<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from faker import Faker 
import altair as alt
import matplotlib.pyplot as plt
# alt.renderers.enable('notebook')
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [21]:
f=Faker()
names = [(f.unique.first_name(),f.unique.last_name()) for i in range(100)]

In [22]:
def create_fake_df():
    df = pd.DataFrame()
    for stage in range(0,3):
        for centrality_name in ['betweenness', 'closeness']:
            df_temp = pd.DataFrame()
            df_temp['name'] = names
            df_temp['stage'] = stage
            df_temp['centrality_name'] = centrality_name
            df_temp['centrality'] = np.random.rand(100)
            df = df.append(df_temp)
    return df

In [23]:
df1 = create_fake_df()
df2 = create_fake_df()

In [24]:
df1

,name,stage,centrality_name,centrality
0,"(Tyler, Kemp)",0,betweenness,0.348390
1,"(Daniel, Brown)",0,betweenness,0.709850
2,"(Leslie, Wright)",0,betweenness,0.893919
3,"(Adrian, Ellis)",0,betweenness,0.593941
4,"(Stephen, Hogan)",0,betweenness,0.334524
...,...,...,...,...
95,"(Craig, Herman)",2,closeness,0.658836
96,"(Carlos, Phelps)",2,closeness,0.086051
97,"(Desiree, Farrell)",2,closeness,0.108722
98,"(Gregory, Johnson)",2,closeness,0.466499


In [25]:
def create_zoomable_bar_chart(df, X, Y, selection, title = '',zoom_encodings = ['x'], top = 100, figsize = (500, 200)):
    '''
    comments
    '''
    brush = alt.selection(type="interval", encodings=zoom_encodings)

    base = alt.Chart(df, title="Select a range in the base view below").mark_bar().encode(x=alt.X( X+':O', sort='-y'),
        y=Y+':Q'
    ).add_selection(
        brush
    ).transform_filter(
        selection
    ).properties(
        width=figsize[0],
        height=figsize[1]*0.1
    )   
    
    zoomed = alt.Chart(df, title=title).mark_bar().transform_filter(
        selection
    ).transform_filter(
        brush
    ).encode(x=alt.X(X+':O', sort='-y'),
        y=Y+':Q'
    ).properties(
        width=figsize[0],
        height=figsize[1]*0.9
    )
    
    if top:
        base.transform_window(
            rank='rank('+Y+')',
            sort=[alt.SortField(Y, order='descending')]
        ).transform_filter(
            (alt.datum.rank < top)
        )
        zoomed.transform_window(
            rank='rank('+Y+')',
            sort=[alt.SortField(Y, order='descending')]
        ).transform_filter(
            (alt.datum.rank < top)
        )
        
    chart = zoomed & base
    
    return chart

In [26]:
centrality_options = df1["centrality_name"].unique().tolist()
dropdown = alt.binding_select(options=centrality_options)
slider1 = alt.binding_range(min=0, max=max(df1["stage"]), step=1)

selection = alt.selection_single(
    fields=['centrality_name', "stage"],
    bind={'centrality_name': dropdown, 'stage': slider1},
    name="Select",
    init={"centrality_name": "betweenness", "stage": 0}
)

In [27]:
chart1 = create_zoomable_bar_chart(df1,'name','centrality',selection, title = 'Top 100 nodes with highest centrality (Undirected)')
chart2 = create_zoomable_bar_chart(df2,'name','centrality',selection, title = 'Top 100 nodes with highest centrality (directed)')
chart = chart1 | chart2
chart.add_selection(
    selection
)


alt.HConcatChart(...)